In [1]:
import pandas as pd

resenha = pd.read_csv('dados/imdb-reviews-pt-br.csv')

In [2]:
resenha.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [3]:
resenha['classificacao'] = resenha.sentiment.map({'neg': 0, 'pos': 1})
resenha.head()

,id,text_en,text_pt,sentiment,classificacao
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,0
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,0
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,0
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,0
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,0


**Feature Extraction** transformar o texto em uma informação numérica.

**Bag of Words** é uma forma de representar o texto de acordo com a ocorrência das palavras nele.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

texto = ["Assisti um filme ótimo", "Assisti um filme ruim"]
vetorizar = CountVectorizer(lowercase=False)
bag_of_words = vetorizar.fit_transform(texto)

In [5]:
vetorizar.get_feature_names_out()

array(['Assisti', 'filme', 'ruim', 'um', 'ótimo'], dtype=object)

### Matriz esparsa
Uma matriz é dita esparsa quando possui uma grande quantidade de elementos com valor zero[1] (ou não presentes, ou não necessários).

In [6]:
# bag_of_words retorna uma matriz esparsa
# scipy.sparse
bag_of_words

<2x5 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>

- Converter uma matriz esparsa em um dataframe pandas

In [7]:
matriz_esparsa = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns=vetorizar.get_feature_names_out())

In [8]:
# max_features define a quantidade de colunas do dataframe levando em consideração as palavras mais frenquentes
vetorizar = CountVectorizer(lowercase=False, max_features=50)
bags_of_words = vetorizar.fit_transform(resenha.text_pt)
print(bags_of_words.shape)

(49459, 50)


In [9]:
from sklearn.model_selection import train_test_split

treino, teste, classe_treino, classe_teste = train_test_split(bags_of_words, resenha.classificacao, random_state=42)

In [10]:
from sklearn.linear_model import LogisticRegression

regressao_ligistica = LogisticRegression()
regressao_ligistica.fit(treino, classe_treino)
acuracia = regressao_ligistica.score(teste, classe_teste)

In [11]:
acuracia

0.6583097452486858

### Funçao para classificar

In [12]:
def classificar_texto(corpus, coluna_texto, coluna_classificacao):
    vetorizar = CountVectorizer(lowercase=False, max_features=50)
    bags_of_words = vetorizar.fit_transform(corpus[coluna_texto])
    
    x_treino, x_teste, y_treino, y_teste = train_test_split(bags_of_words, 
                                                            corpus[coluna_classificacao],
                                                            random_state=42)
    
    regressao_ligistica = LogisticRegression()
    regressao_ligistica.fit(x_treino, y_treino)
    return regressao_ligistica.score(x_teste, y_teste)

In [13]:
classificar_texto(resenha, 'text_pt', 'classificacao')

0.6583097452486858